In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import silhouette_score, adjusted_rand_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram, linkage


# Загрузка данных
df = pd.read_csv("../data/prepared_data_for_clusterization.csv")

# Признаки (без категорий, как в вашем DBSCAN-коде)
feature_columns = [
    'duration', 'frame rate', 'pixel_count',
    'title_len', 'uppercase_ratio', 'emoji_count', 'has_exclamation',
    'category_Howto & Style', 'category_Music', 'category_News & Politics',
    'category_Nonprofits & Activis', 'category_People & Blogs',
    'category_Pets & Animals', 'category_Science & Technology',
    'category_Shows', 'category_Sports', 'category_Travel & Events',
    'description', 'hashtags'
]


X = df[feature_columns].values

# Числовые и категориальные признаки
numeric = ['duration', 'pixel_count', 'title_len', 'uppercase_ratio', 'emoji_count', 'has_exclamation']
cat_cols = [c for c in df.columns if c.startswith('category_')]

# ───────────────────────────────────────────────────────
# 1. ПОДБОР ПАРАМЕТРОВ ИЕРАРХИЧЕСКОЙ КЛАСТЕРИЗАЦИИ
# ───────────────────────────────────────────────────────

# Параметры
linkage_method = 'ward'  # 'ward', 'average', 'complete', 'single'
metric = 'euclidean'     # для 'ward' — только 'euclidean'

# Подбор K по силуэтному коэффициенту (аналогично вашему K-Means)
K_range = range(2, 11)
best_k = 2
best_sil = -1

print("Подбор K по силуэтному коэффициенту:")
for k in K_range:
    try:
        hc = AgglomerativeClustering(
            n_clusters=k,
            linkage=linkage_method,
            metric=metric if linkage_method != 'ward' else 'euclidean'  # ward игнорирует metric
        )
        labels = hc.fit_predict(X)
        sil = silhouette_score(X, labels, metric='euclidean')
        print(f"  K={k:2d} → Silhouette = {sil:.4f}")
        if sil > best_sil:
            best_sil = sil
            best_k = k
    except Exception as e:
        print(f"  K={k:2d} → Ошибка: {e}")
        continue

print(f"\nВыбрано K = {best_k} (макс. силуэт = {best_sil:.4f})")

# Обучение финальной модели
hc_final = AgglomerativeClustering(
    n_clusters=best_k,
    linkage=linkage_method,
    metric=metric if linkage_method != 'ward' else 'euclidean'
)
cluster_labels = hc_final.fit_predict(X)
df['cluster'] = cluster_labels

n_clusters = len(set(cluster_labels))
n_noise = 0  # Иерархическая кластеризация не выделяет шум

print(f"\nОбучение завершено:")
print(f"  → Найдено кластеров: {n_clusters}")
print(f"  → Точек-выбросов (шум): {n_noise} (0.0%)")

# ───────────────────────────────────────────────────────
# 2. a. ЭКСПЕРТНАЯ ОЦЕНКА
# ───────────────────────────────────────────────────────
print("\n" + "="*50)
print("a. ЭКСПЕРТНАЯ ОЦЕНКА")
print("="*50)

# Распределение по кластерам
print("\nРаспределение объектов по кластерам:")
labels_series = pd.Series(cluster_labels)
cluster_counts = labels_series.value_counts().sort_index()
for label in sorted(cluster_counts.index):
    cnt = cluster_counts[label]
    print(f"  Кластер {label}: {cnt} видео ({cnt / len(df) * 100:.1f}%)")

# Средние значения по кластерам
print("\nСредние значения числовых признаков по кластерам:")
cluster_means = df.groupby('cluster')[numeric].mean().round(3)
print(cluster_means.to_string())

# Доминирующие категории в кластерах
print("\nДоля категорий в кластерах (топ-3 на кластер):")
cat_means = df.groupby('cluster')[cat_cols].mean()
for k in range(n_clusters):
    print(f"\nКластер {k}:")
    top_cats = cat_means.loc[k].nlargest(3)
    for cat, prob in top_cats.items():
        cat_name = cat.replace('category_', '')
        print(f"  • {cat_name:<25} → {prob:.3f}")

# ───────────────────────────────────────────────────────
# 3. b. СРАВНЕНИЕ С РЕАЛЬНЫМИ РАЗБИЕНИЯМИ
# ───────────────────────────────────────────────────────
print("\n" + "="*50)
print("b. СРАВНЕНИЕ С РЕАЛЬНЫМИ РАЗБИЕНИЯМИ")
print("="*50)

# Восстановление истинной категории
df['true_category'] = df[cat_cols].idxmax(axis=1).str.replace('category_', '')
ari_cat = adjusted_rand_score(df['true_category'], df['cluster'])
print(f"ARI (кластеры vs категории): {ari_cat:.4f}")

# Сравнение с популярностью (если есть 'views')
if 'views' in df.columns:
    y_pop = (df['views'] > 200).astype(int)
    ari_pop = adjusted_rand_score(y_pop, df['cluster'])
    print(f"ARI (кластеры vs популярность, views>200): {ari_pop:.4f}")
else:
    print("Столбец 'views' отсутствует — сравнение с популярностью невозможно.")

# ───────────────────────────────────────────────────────
# 4. c. ВИЗУАЛИЗАЦИЯ
# ───────────────────────────────────────────────────────
print("\n" + "="*50)
print("c. ВИЗУАЛИЗАЦИЯ")
print("="*50)
print("Генерация диаграмм...")

# PCA для 2D-проекции
pca = PCA(n_components=2, random_state=42)
X_pca = pca.fit_transform(X)

plt.figure(figsize=(18, 6))



# 2. Топ-3 категории по кластерам
plt.subplot(1, 3, 2)
clusters = list(range(n_clusters))
x = np.arange(len(clusters))
width = 0.8 / 3
colors = plt.cm.tab10.colors

for i in range(3):
    values = []
    for k in clusters:
        top_cats = cat_means.loc[k].nlargest(3)
        values.append(top_cats.iloc[i] if i < len(top_cats) else 0)
    label = f"Топ-{i+1}"
    if i == 0 and n_clusters > 0:
        first_cat = cat_means.loc[clusters[0]].nlargest(1).index[0].replace('category_', '')
        label += f" (напр., '{first_cat}')"
    plt.bar(x + i * width - 0.4 + width/2, values, width,
            label=label, color=colors[i % len(colors)], alpha=0.8)

plt.xlabel('Кластер')
plt.ylabel('Доля видео')
plt.title('Топ-3 категории в кластерах')
plt.xticks(x, [f'Кл.{k}' for k in clusters])
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)

# 3. 2D-проекция (PCA)
plt.subplot(1, 3, 3)
scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1],
                      c=cluster_labels, cmap='tab20', s=25, alpha=0.7,
                      edgecolors='k', linewidth=0.3)
plt.colorbar(scatter, label='Кластер')
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} дисп.)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} дисп.)')
plt.title(f'Иерархическая кластеризация ({linkage_method})\nK = {n_clusters}')

plt.tight_layout()
plt.savefig("hierarchical_result.png", dpi=300, bbox_inches='tight')


plt.show()